In [7]:
import torchaudio
from torch.utils.data import random_split

In [3]:
data_path = 'data'

In [9]:
dataset = torchaudio.datasets.YESNO(root=data_path, download=True)

In [10]:
# Define the sizes for the training and testing sets
train_size = int(0.8 * len(dataset))  # 80% for training
test_size = len(dataset) - train_size  # 20% for testing

# Split the dataset into training and testing sets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Now you have train_dataset and test_dataset ready for use
print(f"Training set size: {len(train_dataset)}")
print(f"Testing set size: {len(test_dataset)}")


Training set size: 48
Testing set size: 12


In [13]:
train_dataset[0][0].shape

torch.Size([1, 43200])